In [1]:
# Importing the required libraries 
import PyPDF2
import gtts
from playsound import playsound
import fitz
import json
import re
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from gtts import gTTS

In [2]:

## To get the median
def helper_median(lst):
    sortedLst = sorted(lst)
    lstLen = len(lst)
    index = (lstLen - 1) // 2

    if (lstLen % 2):
        return sortedLst[index]
    else:
        return (sortedLst[index] + sortedLst[index + 1])/2.0

# To get the font median
def font_analysis(filename):
    doc = fitz.open(filename)
    font_size = []
    font = []

    for page in doc:
        sizes =[]
        texts = []
        bold = []

        displayList = page.getDisplayList()

        textPage = displayList.getTextPage()

        getJsonStr = textPage.extractJSON()

        getJson = json.loads(getJsonStr)

        for block in getJson['blocks']:
            for line in block['lines']:
                for span in line['spans']:
                    font_size.append(span['size'])
                    font.append(span['font'])

    median_size = helper_median(font_size)

    return median_size


# Converting the pdf to text 
def pdfToText(filename):
    doc = fitz.open(filename)
    allText = ''
    count = 1
    pageNo = 1
    for page in doc:
        sizes =[]
        texts = []
        bold = []

        displayList = page.getDisplayList()

        textPage = displayList.getTextPage()

        getJsonStr = textPage.extractJSON()

        getJson = json.loads(getJsonStr)

        for block in getJson['blocks']:
            for line in block['lines']:
                allText =  allText + '\n'
                for span in line['spans']:
                    sentence = span['text']
                    font_size = span['size']
                    font = span['font'].lower()
                    
                    if sentence.replace(" ", "").isdigit():
                        continue
                    
#                     font_median = font_analysis(filename)
                    
#                     if font_size > 20 and 'bold' in font:
#                         allText += f'Title {count}' + '\n' + sentence
#                         count += 1
#                         continue
                        
                    allText += sentence
        pageNo += 1
        
    return allText

In [63]:
import re
doc = fitz.open("sample_pdfs/rev-cnn-dac2018.pdf")
allText = ''
count = 1
pageNo = 1
# for page in doc:

page = doc[2]
sizes =[]
texts = []
bold = []

displayList = page.getDisplayList()

textPage = displayList.getTextPage()

getJsonStr = textPage.extractJSON()

getJson = json.loads(getJsonStr)

for block in getJson['blocks']:
    for line in block['lines']:
        allText =  allText + '\n'
        for span in line['spans']:
            print(len(line['spans']))
            print(span)
            sentence = span['text']
            font_size = span['size']
            font = span['font'].lower()
                

            if sentence.replace(" ", "").isdigit():
                continue
            
            allText += sentence

                
# print(allText)

1
{'size': 8.876283645629883, 'flags': 4, 'font': 'LinLibertineT', 'color': 0, 'text': 'in the neural network are explicitly revealed by the memory access', 'origin': [53.79800033569336, 93.64898681640625], 'bbox': [53.79800033569336, 83.56178283691406, 294.0453796386719, 95.95335388183594]}
1
{'size': 9.055620193481445, 'flags': 4, 'font': 'LinLibertineT', 'color': 0, 'text': 'pattern. For example, a CNN performs an activation operation', 'origin': [53.79800033569336, 104.05999755859375], 'bbox': [53.79800033569336, 93.97279357910156, 294.0470886230469, 106.36436462402344]}
1
{'size': 8.979839324951172, 'flags': 4, 'font': 'LinLibertineT', 'color': 0, 'text': 'after each convolution followed by an optional pooling operation.', 'origin': [53.79800033569336, 114.47100830078125], 'bbox': [53.79800033569336, 104.38380432128906, 295.42999267578125, 116.77537536621094]}
1
{'size': 9.020037651062012, 'flags': 4, 'font': 'LinLibertineT', 'color': 0, 'text': 'These three operations are often m

In [3]:
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing


## Selecting the file location
filelocation = askopenfilename() # open the dialog GUI

allText = pdfToText(filelocation)
print(allText)
# with open(filelocation, "rb") as f:  # open the file in reading (rb) mode and call it f
#     pdf = pdftotext.PDF(f)  # store a text version of the pdf file f in pdf variable

# string_of_text = ''
# for text in pdf:
#     string_of_text += text
# from tqdm import tqdm

# final_file = gTTS(text=Alltext, lang='en', slow = False)  # store file in variable
# final_file.save("Generated Speech.mp3")  


Reverse Engineering Convolutional Neural Networks Through
Side-channel Information Leaks
Weizhe Hua, Zhiru Zhang, and G. Edward Suh
School of Electrical and Computer Engineering, Cornell University, Ithaca, NY
{wh399, zhiruz, gs272}@cornell.edu
ABSTRACT
A convolutional neural network (CNN) model represents a crucial
piece of intellectual property in many applications. Revealing its
structure or weights would leak con￿dential information. In this pa-
per we present novel reverse-engineering attacks on CNNs running
on a hardware accelerator, where an adversary can feed inputs to
the accelerator and observe the resulting o￿-chip memory accesses.
Our study shows that even with data encryption, the adversary can
infer the underlying network structure by exploiting the memory
and timing side-channels. We further identify the information leak-
age on the values of weights when a CNN accelerator performs
dynamic zero pruning for o￿-chip memory accesses. Overall, this
work reveals the importan

In [21]:
import copy 

#calculating the iteration fro alltext by 4800 (as google vision can take only 5000 per request)
tot_iteration, tot_division = len(allText)//4800, len(allText)/4800

# Partitioning it to total_text/4800 = no. of files
start = 0 
end = 4800
count = 1
for i in range(0, tot_iteration):
    globals()['part%s' % count] = allText[start:end]
    count += 1
    start = copy.deepcopy(end) 
    end = end + 4800
    

leftover = int(4800*(tot_division - tot_iteration))
final_end = start + leftover

globals()['part%s' % count] = allText[start: end]


7.5475
0
4800
4800
9600
9600
14400
14400
19200
19200
24000
24000
28800
28800
33600
final_start 33600
final_end 36228


In [34]:
# check = ''
# for i in range(1, tot_iteration+2):
#     print(i)
#     check += globals()['part%s' % i]



1
2
3
4
5
6
7
8

Reverse Engineering Convolutional Neural Networks Through
Side-channel Information Leaks
Weizhe Hua, Zhiru Zhang, and G. Edward Suh
School of Electrical and Computer Engineering, Cornell University, Ithaca, NY
{wh399, zhiruz, gs272}@cornell.edu
ABSTRACT
A convolutional neural network (CNN) model represents a crucial
piece of intellectual property in many applications. Revealing its
structure or weights would leak con￿dential information. In this pa-
per we present novel reverse-engineering attacks on CNNs running
on a hardware accelerator, where an adversary can feed inputs to
the accelerator and observe the resulting o￿-chip memory accesses.
Our study shows that even with data encryption, the adversary can
infer the underlying network structure by exploiting the memory
and timing side-channels. We further identify the information leak-
age on the values of weights when a CNN accelerator performs
dynamic zero pruning for o￿-chip memory accesses. Overall, this
work reve

In [21]:
from google.cloud import texttospeech
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/hiteshsaaimanancherypanneerselvam/Documents/Development_Projects/pdf_to_audiobook/My First Project-1f1b9b4bc545.json"


# Instantiates a client
client = texttospeech.TextToSpeechClient()

# Set the text input to be synthesized
synthesis_input = texttospeech.SynthesisInput(text= allText)

# Build the voice request, select the language code ("en-US") and the ssml
# voice gender ("neutral")
voice = texttospeech.VoiceSelectionParams(
    language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
)

# Select the type of audio file you want returned
audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)

# Perform the text-to-speech request on the text input with the selected
# voice parameters and audio file type
response = client.synthesize_speech(
    input=synthesis_input, voice=voice, audio_config=audio_config
)

# The response's audio_content is binary.
with open("output.mp3", "wb") as out:
    # Write the response to the output file.
    out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')

InvalidArgument: 400 5000 characters limit exceeded.

In [42]:
print(allText)



Reverse Engineering Convolutional Neural Networks Through
Side-channel Information Leaks
Weizhe Hua, Zhiru Zhang, and G. Edward Suh
School of Electrical and Computer Engineering, Cornell University, Ithaca, NY
{wh399, zhiruz, gs272}@cornell.edu
ABSTRACT
A convolutional neural network (CNN) model represents a crucial
piece of intellectual property in many applications. Revealing its
structure or weights would leak con￿dential information. In this pa-
per we present novel reverse-engineering attacks on CNNs running
on a hardware accelerator, where an adversary can feed inputs to
the accelerator and observe the resulting o￿-chip memory accesses.
Our study shows that even with data encryption, the adversary can
infer the underlying network structure by exploiting the memory
and timing side-channels. We further identify the information leak-
age on the values of weights when a CNN accelerator performs
dynamic zero pruning for o￿-chip memory accesses. Overall, this
work reveals the importan

In [43]:
print(check)


Reverse Engineering Convolutional Neural Networks Through
Side-channel Information Leaks
Weizhe Hua, Zhiru Zhang, and G. Edward Suh
School of Electrical and Computer Engineering, Cornell University, Ithaca, NY
{wh399, zhiruz, gs272}@cornell.edu
ABSTRACT
A convolutional neural network (CNN) model represents a crucial
piece of intellectual property in many applications. Revealing its
structure or weights would leak con￿dential information. In this pa-
per we present novel reverse-engineering attacks on CNNs running
on a hardware accelerator, where an adversary can feed inputs to
the accelerator and observe the resulting o￿-chip memory accesses.
Our study shows that even with data encryption, the adversary can
infer the underlying network structure by exploiting the memory
and timing side-channels. We further identify the information leak-
age on the values of weights when a CNN accelerator performs
dynamic zero pruning for o￿-chip memory accesses. Overall, this
work reveals the importan